### Goal: 
Create function that given some "ideal conditons" can spit out a dataframe with `Ideal Wind`, `Ideal Period`, `Ideal Tide`, and `Ideal Swell` columns. <br/>
This is key because will likely want to change ideal parameters depending on new information/would be interesting to run with different numbers anyways.

In [6]:
import pandas as pd
import numpy as np
from pathlib import Path

In [24]:
root_folder = Path.cwd().parents[1]

df = pd.read_csv(root_folder/'data/processed/00-final.csv')

In [20]:
def ideal_wind(df, wind_direction: list, wind_speed: list):
    """Creates Ideal Wind column, 0 for bad wind, 1 for good wind"""

    zeros = np.zeros(df.shape[0],dtype=int)
    df['Ideal Wind']=zeros

    #low speed winds are ideal
    df.loc[df['Wind Speed'] <= wind_speed[0], 'Ideal Wind'] = 1

    #filter for direction
    wdints = pd.to_numeric(df['Wind Direction'],errors='coerce') #only care about integer values
    goodwind = (wind_direction[0] <= wdints) & (wdints <= wind_direction[1]) 
    df.loc[goodwind, 'Ideal Wind'] = 1

    #high speed winds bad
    df.loc[df['Wind Speed']>=wind_speed[1], 'Ideal Wind'] = 0

    return


In [6]:
df.head()

,PST,Tide,Height,Deg,Period,Wind Speed,Wind Direction
0,2017-01-01 07:00:00-08:00,3.82,2.47,306.0,11.0,7.0,260
1,2017-01-01 08:00:00-08:00,4.42,2.37,309.0,9.0,6.0,40
2,2017-01-01 09:00:00-08:00,5.13,2.38,310.0,11.0,0.0,000
3,2017-01-01 10:00:00-08:00,5.66,2.66,314.0,11.0,0.0,000
4,2017-01-01 11:00:00-08:00,5.71,2.38,303.0,11.0,5.0,150


In [7]:
df['Period'].value_counts()

4.0       6694
9.0       5233
7.0       3795
11.0      3102
13.0      2652
15.0      1572
17.0       510
20.0       253
970.0        1
1018.0       1
Name: Period, dtype: int64

Since I don't have exact ideal conditions measurements yet I'm just going to use what I think is good for now **but will change later** 

In [21]:
def ideal_period(df, good_period: int):
    "Creates Ideal Period column, 0 for bad period, 1 for good period"
    
    zeros = np.zeros(df.shape[0],dtype=int)
    df['Ideal Period']=zeros
    df.loc[df['Period']>=good_period, 'Ideal Period'] = 1

    return

In [25]:
ideal_period(df, 8)
ideal_wind(df,[0,57],[5,10])

In [26]:
df.head()

,PST,Tide,Height,Deg,Period,Wind Speed,Wind Direction,Ideal Period,Ideal Wind
0,2017-01-01 07:00:00-08:00,3.82,2.47,306.0,11.0,7.0,260,1,0
1,2017-01-01 08:00:00-08:00,4.42,2.37,309.0,9.0,6.0,40,1,1
2,2017-01-01 09:00:00-08:00,5.13,2.38,310.0,11.0,0.0,calm,1,1
3,2017-01-01 10:00:00-08:00,5.66,2.66,314.0,11.0,0.0,calm,1,1
4,2017-01-01 11:00:00-08:00,5.71,2.38,303.0,11.0,5.0,150,1,1


In [27]:
df['Ideal Period'].value_counts()

1    13324
0    10526
Name: Ideal Period, dtype: int64

In [29]:
df['Ideal Wind'].value_counts()

0    15338
1     8512
Name: Ideal Wind, dtype: int64

In [34]:
def ideal_tide(df, good_tide: list):
    """Creates Ideal Tide column, 0 for bad tide, 1 for good tide"""

    zeros = np.zeros(df.shape[0],dtype=int)
    df['Ideal Tide']=zeros

    good = (good_tide[0] <= df.Tide) & (df.Tide <= good_tide[1])
    df.loc[good, 'Ideal Tide'] = 1

    return

In [ ]:
def ideal_swell(df, direction: list, height: list):
    """Creates Ideal Swell column, 0 for bad swell, 1 for good swell"""

    zeros = np.zeros(df.shape[0],dtype=int)
    df['Ideal Swell'] = zeros

    good_direction = (direction[0] <= df.Deg) & (df.Deg <= direction[1])
    good_height = (height[0] <= df.Height) & (df.Height <= height[1])
    good = good_direction & good_height
    df.loc[good, 'Ideal Swell'] = 1

    return

In [ ]:
#making a dictionary for ideal conditions so that I can easily access and change individual items

Ideals = {
    "wind_speed": [5,10],
    "wind_direction": [0,57],
    "tide": [2,5],
    "period": 8,
    "swell_direction": [258,304],
    "swell_height": [1,3]
}